# Basic Motion

In [1]:
import sys
sys.path.insert(0, '/opt/ros/noetic/lib/python3/dist-packages')
import rospy
rospy.init_node('motion_control')

## create an instance of a robot
from jetbot import Bones
robot = Bones()
robot.observe(robot.fwd_cb,'fwdspd')
robot.observe(robot.ang_cb,'angspd')

## create an auto recurring timer and associated callback which triggers ever 250ms. The purpose is to throttle the transfer of twist 
## commands to the arduino.  
from jetbot import Mytimer

def ard_callback():
    robot.twist()

ard_timer = Mytimer(0.25, ard_callback)
ard_timer.start()

In [2]:
import ipywidgets.widgets as widgets
from IPython.display import display

# create two sliders with range [-0.2, 0.2]
left_slider = widgets.FloatSlider(description='left', min=-0.2, max=0.2, step=0.02, orientation='vertical')
right_slider = widgets.FloatSlider(description='right', min=-0.4, max=0.4, step=0.02, orientation='vertical')

# create a horizontal box container to place the sliders next to each other
slider_container = widgets.HBox([left_slider, right_slider])

# display the container in this cell's output
display(slider_container)

Now that we've imported the ``Robot`` class we can initialize the class *instance* as follows. 

In [3]:
import traitlets
left_link = traitlets.dlink((left_slider, 'value'), (robot, 'fwdspd'))
right_link = traitlets.dlink((right_slider, 'value'), (robot, 'angspd'))

In [4]:
robot.stop()

In [ ]:
robot.forward(0.08)

Maybe we only want to run the robot for a set period of time.  For that, we can use the Python ``time`` package.  

In [ ]:
import time

This package defines the ``sleep`` function, which causes the code execution to block for the specified number of seconds
before running the next command.  Try the following to make the robot turn left only for half a second.

You should see the robot move in the same exact way!

### Link motors to traitlets

A really cool feature about these [traitlets](https://github.com/ipython/traitlets) is that we can 
also link them to other traitlets!  This is super handy because Jupyter Notebooks allow us
to make graphical ``widgets`` that use traitlets under the hood.  This means we can attach
our motors to ``widgets`` to control them from the browser, or just visualize the value.

To show how to do this, let's create and display two sliders that we'll use to control our motors.


Now try dragging the sliders (slowly at first).  You should see the respective motor turn!


The ``link`` function that we created above actually creates a bi-directional link!  That means,
if we set the motor values elsewhere, the sliders will update!  Try executing the code block below

In [ ]:
robot.forward(0.1)
time.sleep(2.0)
robot.stop()

You should see the sliders respond to the motor commands!  If we want to remove this connection we can call the
``unlink`` method of each link.

In [ ]:
#left_link.unlink()
#right_link.unlink()

But what if we don't want a *bi-directional* link, let's say we only want to use the sliders to display the motor values,
but not control them.  For that we can use the ``dlink`` function.  The left input is the ``source`` and the right input is the ``target``

Now try moving the sliders.  You should see that the robot doesn't respond.  But when set the motors using a different method,
the sliders will update and display the value!

### Attach functions to events

Another way to use traitlets, is by attaching functions (like ``forward``) to events.  These
functions will get called whenever a change to the object occurs, and will be passed some information about that change
like the ``old`` value and the ``new`` value.  

Let's create and display some buttons that we'll use to control the robot.

In [ ]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

You should see a set of robot controls displayed above!  But right now they wont do anything.  To do that
we'll need to create some functions that we'll attach to the button's ``on_click`` event.  

In [ ]:
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)
    time.sleep(1.0)
    robot.stop()

def step_backward(change):
    robot.backward(0.4)
    time.sleep(1.0)
    robot.stop()

def step_left(change):
    robot.left(0.3)
    time.sleep(1.0)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(1.0)
    robot.stop()

Now that we've defined the functions, let's attach them to the on-click events of each button

In [ ]:
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

Now when you click each button, you should see the robot move!

### Heartbeat Killswitch

Here we show how to connect a 'heartbeat' to stop the robot from moving.  This is a simple way to detect if the robot connection is alive.  You can lower the slider below to reduce the period (in seconds) of the heartbeat.  If a round-trip communication between browser cannot be made within two heartbeats, the '`status`' attribute of the heartbeat will be set ``dead``.  As soon as the connection is restored, the ``status`` attribute will return to ``alive``.

In [ ]:
from jetbot import Heartbeat

heartbeat = Heartbeat()

# this function will be called when heartbeat 'alive' status changes
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        robot.stop()
        
heartbeat.observe(handle_heartbeat_status, names='status')

period_slider = widgets.FloatSlider(description='period', min=0.001, max=0.5, step=0.01, value=0.5)
traitlets.dlink((period_slider, 'value'), (heartbeat, 'period'))

display(period_slider, heartbeat.pulseout)

Try executing the code below to start the motors, and then lower the slider to see what happens.  You can also try disconnecting your robot or PC.

In [ ]:
# robot.left(0.2) 

# now lower the `period` slider above until the network heartbeat can't be satisfied

### Conclusion

That's it for this example notebook!  Hopefully you feel confident that you can program your robot to move around now :)